In [1]:
from sowce.data.dataset_classes import VisiumCohort
from loguru import logger
import pandas as pd
import numpy as np
import os
import random

In [6]:
dataset_id = "dataset-dc3510e2-76eb-47f6-9bb4-d20d340ee808"
zarr_dir = f"/home/owkin/data/{dataset_id}/without_spotclean/SpatialData"
path_dataset = f"{zarr_dir}/*.zarr"
dataset = VisiumCohort(
    path_dataset=path_dataset,
    normalization="log_norm_count", # load log(1+CPM) data
    deconvolution_slot_name = "deconv_cohort_level_2",
    radius=1,
    ignore_sample_ids=["CH_B_015a_vis_Visium"], # We need to ignore this samples that failed QC to avoid an error in dataset.filter_umi() later
    list_sample_ids=[ids_intersection[0]]
)
_ = dataset.load_counts_cohort()
_ = dataset.load_deconvolution_cohort() # include cell type mapping in this function if needed
logger.info("Filtering UMI...")
dataset.filter_umi(200)
logger.info("Filtering done.")

2024-10-31 16:00:09.787 | INFO     | __main__:<module>:14 - Filtering UMI...
2024-10-31 16:00:16.259 | INFO     | __main__:<module>:16 - Filtering done.


In [7]:
sample = dataset._dict_visium_samples[ids_intersection[0]]
sample.list_celltypes

['B_cell',
 'DC',
 'Endothelial',
 'Fibroblast',
 'Granulocyte',
 'Malignant_bladder',
 'Mast',
 'MoMac',
 'Plasma',
 'Smooth_muscle',
 'T_NK']

In [2]:
ids_visium = ['CH_B_001a_vis',
 'CH_B_001b_vbu',
 'CH_B_002a_vis',
 'CH_B_002b_vbu',
 'CH_B_003b_vbu',
 'CH_B_004a_vis',
 'CH_B_004b_vbu',
 'CH_B_006a_vis',
 'CH_B_006b_vis',
 'CH_B_007a_vis',
 'CH_B_008a_vis',
 'CH_B_008b_vis',
 'CH_B_008b_vis16PCR',
 'CH_B_008b_vis22PCR',
 'CH_B_009a_vis',
 'CH_B_009b_vis',
 'CH_B_010a_vis',
 'CH_B_010b_vis',
 'CH_B_011a_vis',
 'CH_B_011b_vis',
 'CH_B_012a_vis',
 'CH_B_013a_vis',
 'CH_B_014a_vis',
 'CH_B_015a_vis',
 'CH_B_016a_vis',
 'CH_B_017a_vis',
 'CH_B_018a_vis',
 'CH_B_019a_vis',
 'CH_B_020a_vis',
 'CH_B_021a_vis',
 'CH_B_022a_vis',
 'CH_B_022b_vis',
 'CH_B_023a_vbu',
 'CH_B_023a_vis',
 'CH_B_024a_vis',
 'CH_B_025a_vis',
 'CH_B_025b_vis',
 'CH_B_026a_vis',
 'CH_B_026b_vis',
 'CH_B_027a_vis',
 'CH_B_028a_vis',
 'CH_B_029a_vis',
 'CH_B_030a_vis',
 'CH_B_031a_vis',
 'CH_B_032a_vis',
 'CH_B_033a_vbu',
 'CH_B_034a_vis',
 'CH_B_035a_vis',
 'CH_B_036a_vis',
 'CH_B_037a_vis',
 'CH_B_039a_vis',
 'CH_B_041a_vbu',
 'CH_B_044a_vbu',
 'CH_B_045a_vis',
 'CH_B_046a_vis',
 'CH_B_047a_vbu',
 'CH_B_050a_vis',
 'CH_B_051a_vis',
 'CH_B_052a_vis',
 'CH_B_053a_vbu',
 'CH_B_054a_vbu',
 'CH_B_055a_vis',
 'CH_B_056a_vis',
 'CH_B_057a_vis',
 'CH_B_058a_vis',
 'CH_B_059a_vis',
 'CH_B_060a_vis',
 'CH_B_062a_vis',
 'CH_B_063a_vis',
 'CH_B_064a_vis',
 'CH_B_066a_vis',
 'CH_B_067a_vis',
 'CH_B_068a_vis',
 'CH_B_069a_vis',
 'CH_B_070a_vis',
 'CH_B_071a_vis',
 'CH_B_072a_vis',
 'CH_B_073a_vis',
 'CH_B_074a_vis',
 'CH_B_075a_vis',
 'CH_B_076a_vis',
 'CH_B_077a_vis',
 'CH_B_078a_vis',
 'CH_B_079a_vis',
 'CH_B_080a_vis',
 'CH_B_082a_vbu',
 'CH_B_082a_vis',
 'CH_B_083a_vis',
 'CH_B_084a_vis',
 'CH_B_085a_vis',
 'CH_B_086a_vis',
 'CH_B_087a_vis',
 'CH_B_088a_vis',
 'CH_B_090a_vis',
 'CH_B_091a_vis',
 'CH_B_092a_vis',
 'CH_B_093a_vis',
 'CH_B_094a_vis',
 'CH_B_095a_vis',
 'CH_B_096a_vis',
 'CH_B_097a_vis']
ids_visium_short = [x[:-4] for x in ids_visium]

In [3]:
fpath = "/home/owkin/project/USR_DATA/DATA_YBARECHE/RNAXplorer_analysis/RNAXplorer_CHUV_Bladder_Baseline-patids.csv"
df_baseline = pd.read_csv(fpath)

discovery_clinical = pd.read_csv("/home/owkin/project/mosaic_data/20240724/mosaic-chuv.uker-mibc-1.3_postruv_clinical_data.csv")
#discovery_clinical = discovery_clinical[discovery_clinical['os_censor'] == 0]
merged_df = pd.merge(df_baseline, discovery_clinical, left_on="Tumour block ID for MOSAIC", right_on="patient_id")
ids_clinical = merged_df["Tumour block ID for MOSAIC"].tolist()

In [4]:
ids_intersection = []
for elem in ids_visium:
    if elem[:-4] in ids_clinical:
        ids_intersection.append(elem)
len(ids_intersection)
merged_df = merged_df[merged_df["Tumour block ID for MOSAIC"].isin(ids_visium_short)]
merged_df["visium_id"] = [
    elem + "_vis" if elem + "_vis" in ids_visium else elem + "_vbu" 
    for elem in merged_df["Tumour block ID for MOSAIC"]
]
med = merged_df["os_years"].median()
len(ids_intersection)

76

In [5]:
ids_intersection

['CH_B_001b_vbu',
 'CH_B_002b_vbu',
 'CH_B_003b_vbu',
 'CH_B_004b_vbu',
 'CH_B_006b_vis',
 'CH_B_008b_vis',
 'CH_B_009b_vis',
 'CH_B_010a_vis',
 'CH_B_011a_vis',
 'CH_B_013a_vis',
 'CH_B_016a_vis',
 'CH_B_019a_vis',
 'CH_B_020a_vis',
 'CH_B_022a_vis',
 'CH_B_023a_vbu',
 'CH_B_023a_vis',
 'CH_B_025a_vis',
 'CH_B_027a_vis',
 'CH_B_029a_vis',
 'CH_B_030a_vis',
 'CH_B_031a_vis',
 'CH_B_032a_vis',
 'CH_B_033a_vbu',
 'CH_B_034a_vis',
 'CH_B_035a_vis',
 'CH_B_036a_vis',
 'CH_B_037a_vis',
 'CH_B_039a_vis',
 'CH_B_041a_vbu',
 'CH_B_044a_vbu',
 'CH_B_045a_vis',
 'CH_B_046a_vis',
 'CH_B_047a_vbu',
 'CH_B_050a_vis',
 'CH_B_052a_vis',
 'CH_B_053a_vbu',
 'CH_B_054a_vbu',
 'CH_B_055a_vis',
 'CH_B_056a_vis',
 'CH_B_057a_vis',
 'CH_B_058a_vis',
 'CH_B_059a_vis',
 'CH_B_060a_vis',
 'CH_B_062a_vis',
 'CH_B_063a_vis',
 'CH_B_064a_vis',
 'CH_B_066a_vis',
 'CH_B_067a_vis',
 'CH_B_068a_vis',
 'CH_B_069a_vis',
 'CH_B_070a_vis',
 'CH_B_071a_vis',
 'CH_B_072a_vis',
 'CH_B_073a_vis',
 'CH_B_074a_vis',
 'CH_B_075

In [11]:
# Save sample names
with open("/home/owkin/project/cytocommunity_results/chuv/ImageNameList.txt", "w") as file:
    for item in merged_df["visium_id"]:
        file.write(f"{item}\n")

In [12]:
# Save most dominant cell type
#for id_visium in merged_df["visium_id"]:
#    sample = dataset._dict_visium_samples[id_visium]
#    deconvolution_frac = sample.deconvolution_frac
#    max_indices = np.argmax(deconvolution_frac, axis=1)
#    dominant_cell_types = [sample.list_celltypes[i] for i in max_indices]
#    with open(f"supervised/chuv/{id_visium}_CellTypeLabel.txt", "w") as file:
#        for item in dominant_cell_types:
#            file.write(f"{item}\n")

In [13]:
# Save deconvolution fraction
for id_visium in merged_df["visium_id"]:
    sample = dataset._dict_visium_samples[id_visium]
    deconvolution_frac = sample.deconvolution_frac
    np.save(f"/home/owkin/project/cytocommunity_results/chuv/{id_visium}_DeconvolutionFrac.npy", deconvolution_frac)

In [14]:
for id_visium in merged_df["visium_id"]:
    sample = dataset._dict_visium_samples[id_visium]
    x = sample.neighbors_idx.index.get_level_values("x").tolist()
    y = sample.neighbors_idx.index.get_level_values("y").tolist()
    with open(f"/home/owkin/project/cytocommunity_results/chuv/{id_visium}_Coordinates.txt", "w") as file:
        for elem1, elem2 in zip(x, y):
            file.write(f"{elem1}\t {elem2}\n")

In [19]:
# Dummy labels
for id_visium in merged_df["visium_id"]:
    with open(f"/home/owkin/project/cytocommunity_results/unsupervised/chuv/{id_visium}_GraphLabel.txt", "w") as file:
        file.write(str(random.randint(0, 1)))

In [85]:
ids_low = merged_df[merged_df["os_years"] > med]["visium_id"]
for id_visium in ids_low:
    with open(f"supervised/chuv/{id_visium}_GraphLabel.txt", "w") as file:
        file.write("0")

ids_high = merged_df[merged_df["os_years"] <= med]["visium_id"]
for id_visium in ids_high:
    with open(f"supervised/chuv/{id_visium}_GraphLabel.txt", "w") as file:
        file.write("1")

In [3]:
len(os.listdir("/home/owkin/project/cytocommunity_results/chuv"))

74

In [13]:
import os

def count_elements_in_folder(folder_path):
    try:
        # List all elements in the folder
        elements = os.listdir(folder_path)
        # Count the number of elements
        return len(elements)
    except FileNotFoundError:
        print("The specified folder does not exist.")
        return 0

# Example usage
folder_path = '/home/owkin/data/dataset-dc3510e2-76eb-47f6-9bb4-d20d340ee808/spaceranger_count'
print("Number of elements in folder:", count_elements_in_folder(folder_path))


Number of elements in folder: 104
